In [1]:
%matplotlib widget
%pylab
import matplotlib.pyplot as plt
import time
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy.coordinates import AltAz
from astropy.time import TimeDelta
from astropy.coordinates import get_sun
from functools import reduce
import pandas as pd
from tqdm import tqdm

Using matplotlib backend: module://ipympl.backend_nbagg
Populating the interactive namespace from numpy and matplotlib


# Simulation Functions

This notebook has all the functions used for making the $\textbf{Simulations of the Radio Averaged Sky}$.

The array from the Haslam Temperature Map (Haslam et al., 1982) must be called $\textbf{T_Hmap}$ for the code to work.  


### Contents:
1. [Frequency extrapolation](#T_freq)
2. [Antenna gain files](#Gainfiles)
3. [G Matrix](#GMatrix)
4. [Dynamic Spectra](#DS)
5. [Dynamic Spectra with gain pattern](#DSG)
6. [Dynamic Spectra for different latitudes](#DSGLat)
7. [Best times for observing](#TObs)
8. [Galactic Coordinates of the Cenit](#GalCoord)
9. [Simulation of the Sky Temperature extrapolated to a frequency](#SimTsky)

## 1. Frequency extrapolation <a class="anchor" id="T_freq"></a>

The Temperature spectral index is defined by:
    
$$\alpha=-\frac{log(T_1/T_2)}{log(\nu_1/\nu_2)}$$

The function $\textbf{T_freq}$ will extrapolate the temperature of the sky to a given frequency using this equation.

In [1]:
def T_freq(alpha,freq1,freq2,T1):
    '''Takes spectral index (alpha), initial frequency (freq1) and its tempetature (T1), and
       the desired frequency for extrapolation (freq2). Returns the Temperature extrapolation for that frequency'''
    
    return T1*10**(alpha*log10(freq1/freq2))

## 2. Antenna gain files <a class="anchor" id="Gainfiles"></a>

The Gain files must be in this order:

$$g_{\nu}=g_{\nu}(\theta,\phi)$$

$$g_{\nu}(\theta,\phi)=\begin{pmatrix}
g_{\nu}(\theta=0,\phi=0) & g_{\nu}(\theta=0,\phi=1) & \cdots & g_{\nu}(\theta=0,\phi=359)\\ 
g_{\nu}(\theta=1,\phi=0) & g_{\nu}(\theta=1,\phi=1) & \cdots & g_{\nu}(\theta=1,\phi=359)\\
 \vdots &  \vdots  &\vdots & \vdots  \\ 
g_{\nu}(\theta=90,\phi=0) & g_{\nu}(\theta=90,\phi=1) & \cdots & g_{\nu}(\theta=90,\phi=359)\\
\end{pmatrix}$$


This commented code will help to organize the files if they're not in this order.

In [2]:
#Sort files in right order
#This is only necessary when the files have text or are in the wrong order.

#for i in range (0,len(f)):
#    file='{}{}{}{}'.format(path_B,'\Gain_phi_theta_', f[i], "MHz.csv")
#    x=loadtxt(file, skiprows=1,delimiter=',')   #Theta[0:]
#    #new=concatenate((x[:91,181:],x[:91,2:181]),axis=1)       ###Theta[0,90],phi[0,359]
#    savetxt('{}{}{}.txt'.format(path_B,'\Gain_B_',f[i]), x[:,1:361])

## 3. $G[\nu,\theta,\phi]$ Matrix <a class="anchor" id="GMatrix"></a>

The $G[\nu,\theta,\phi]$ will be a matrix of matrices that will store the Gain Pattern of each point in the sky ($\theta,\phi$) for each frequency $\nu$. This matrix will decrease the time of computation for the Dynamic Spectra.


$$G[\nu,\theta,\phi]=\begin{bmatrix}
\begin{bmatrix}
g_{\nu_0}(\theta,\phi)
\end{bmatrix}\\ 
\begin{bmatrix}
g_{\nu_1}(\theta,\phi)
\end{bmatrix}\\ 
\vdots \\ 
\begin{bmatrix}
g_{\nu_m}(\theta,\phi)
\end{bmatrix}\\ 
\end{bmatrix}$$

Where 

$$g_{\nu_n}(\theta,\phi)=\begin{pmatrix}
g_{\nu}(\theta=0,\phi=0) & g_{\nu}(\theta=0,\phi=1) & \cdots & g_{\nu}(\theta=0,\phi=359)\\ 
g_{\nu}(\theta=1,\phi=0) & g_{\nu}(\theta=1,\phi=1) & \cdots & g_{\nu}(\theta=1,\phi=359)\\
 \vdots &  \vdots  &\vdots & \vdots  \\ 
g_{\nu}(\theta=90,\phi=0) & g_{\nu}(\theta=90,\phi=1) & \cdots & g_{\nu}(\theta=90,\phi=359)\\
\end{pmatrix}$$

The function $\textbf{G_matrix}$ will organize the gain matrix using the antenna files in the right order.

In [3]:
def G_matrix(path,name,size):
    '''Takes the path and name of the Gain File, and the number of files for the frequencies (size).
       Returns the G[nu,theta,phi] matrix, normalized to the maximum value'''

    G_FTP=empty((size,91,360))

    for i in range (size):
        file='{}{}{}.txt'.format(path,name, f[i])
        x=loadtxt(file)
        G_FTP[i]=x

    return G_FTP/amax(G_FTP)    #NORMALIZED to the maximum gain value

## 4. Dynamic Spectra <a class="anchor" id="DS"></a>

Averaged temperature matrix for the dynamic spectra of an Isotropic Antenna.

$$T(\nu,t)=\begin{pmatrix}
T(\nu_1,t_m) & T(\nu_2,t_m) & \cdots & T(\nu_n,t_m)\\ 
 \vdots &  \vdots  &\vdots & \vdots  \\ 
T(\nu_1,t_1) & T(\nu_2,t_1) & \cdots & T(\nu_n,t_1)
\end{pmatrix}$$

The function $\textbf{DS}$ will give us the Averaged temperature matrix for the dynamic spectra of an Isotropic Antenna using the Haslam map $\textbf{T_Hmap}$.

In [3]:
def DS(fs,ts,alpha):  
    '''Takes frequencies (fs) and times in AltAz frame (ts) and returns the Temperature matrix'''
 
    Temps_g=zeros((ts.obstime.size,len(fs)))      #Temperature + Gain Matrix (Empty first)
    
    for i in tqdm(range (ts.obstime.size)):  
    
        aa_coord=g_coord.transform_to(ts[i])      #Galactic Coords of Haslam Map to Altaz Coords
        mask=argwhere(aa_coord.alt>0)             #Only coordinates above the horizon
        T_sky=T_Hmap[mask]                        #Only those temperatures
        
        for j in range (len(fs)): 
            T_f=T_freq(alpha,408,fs[j],T_sky)      #Temperature Extrapolation
            
            Temps_g[i,j]=mean(T_f)                #Mean temperature of the sky

    return Temps_g

## 5. Dynamic Spectra with gain pattern $G[\nu,\theta,\phi]$ <a class="anchor" id="DSG"></a>

The function $\textbf{DS_G}$ will return the Averaged temperature matrix for the Dynamic Spectra of an Antenna with a Gain Pattern given by $G[\nu,\theta,\phi]$.

$$T(\nu,t)=\begin{pmatrix}
T(\nu_1,t_1) & T(\nu_2,t_1) & \cdots & T(\nu_n,t_1)\\ 
 \vdots &  \vdots  &\vdots & \vdots  \\ 
T(\nu_1,t_m) & T(\nu_2,t_m) & \cdots & T(\nu_n,t_m)
\end{pmatrix}$$

Where $T(\nu_n,t_m)$ is the average temperature of the sky for the frequency $\nu_n$ and the time $t_m$, corrected for the beam with the $G[\nu,\theta,\phi]$ matrix.

In [2]:
def DS_G(fs,ts,alfa,G_n):  
    '''Takes frequencies (fs), times in AltAz frame (ts), the spectral index (alpha), and the G[nu,theta,phi] matrix (G_n). 
       Returns the Temperature + Gain matrix (Dinamic Spectra)'''
    
    Temps_g=zeros((ts.obstime.size,len(fs)))      #Temperature + Gain Matrix (Empty first)
    
    for i in tqdm(range (ts.obstime.size)):  
    
        aa_coord=g_coord.transform_to(ts[i])      #Galactic Coords of Haslam Map to Altaz Coords
        mask=argwhere(aa_coord.alt>0)             #Only coordinates above the horizon
        T_sky=T_Hmap[mask]                        #Only those temperatures
        
        Th=np.round((90*u.deg-(aa_coord.alt[mask]))/(1*u.deg))    #Thetas to use in same order as Tsky
        Th=Th.value.astype(int)                                   #Int value (dTheta=1deg)
        
        phi=aa_coord.az[mask]                                     #phi's==azimuth
        phi=phi.value.astype(int)                                 #Int value (dphi=1deg)
        
        for j in range (len(fs)): 
            T_f=T_freq(alfa,408,fs[j],T_sky)                #Temperature Extrapolation
            
            gain=G_n[j,Th,phi]/mean(G_n[j,Th,phi])          #Gains of Th and phi in same order as Tsky  
            
            T_f_g=gain*T_f                                  #New T_sky with respective gain (Beam Correction)
            Temps_g[i,j]=mean(T_f_g)                        #Mean temperature

         
    return Temps_g

## 6. Dynamic Spectra for different latitudes <a class="anchor" id="DSGLat"></a>

For a given time the Dynamic Spectra will change for different latitudes. This function will calculate the Dynamic Spectra corrected by the beam of the sky at a given time for an array of latitudes.

In [6]:
def DS_G_Lat(fs,t,alfa,G_n,Locs):  
    '''Takes frequencies (fs), time (t), the spectral index (alpha), the G[nu,theta,phi] matrix (G_n) and the Locations for
       the Dynamic Spectra. Returns the Temperature + Gain matrix (Dynamic Spectra) for those latitudes.'''

    Temps_g=zeros((Locs.size,len(fs)))      #Temperature + Gain Matrix (Empty first)
    
    for i in tqdm(range (Locs.size)):  
        
        altaz_t = AltAz(location=Locs[i], obstime=t)
    
        aa_coord=g_coord.transform_to(altaz_t)    #Galactic Coords of Haslam Map to Altaz Coords
        mask=argwhere(aa_coord.alt>0)             #Only coordinates above the horizon
        T_sky=T_Hmap[mask]                        #Only those temperatures
        
        Th=np.round((90*u.deg-(aa_coord.alt[mask]))/(1*u.deg))    #Thetas to use in same order as Tsky
        Th=Th.value.astype(int)                                   #Int value (dTheta=1deg)
        
        phi=aa_coord.az[mask]                                     #phi's==azimuth
        phi=phi.value.astype(int)                                 #Int value (dphi=1deg)
        
        for j in range (len(fs)): 
            T_f=T_freq(alfa,408,fs[j],T_sky)                #Temperature Extrapolation
            
            gain=G_n[j,Th,phi]/mean(G_n[j,Th,phi])          #Gains of Th and phi in same order as Tsky  
            
            T_f_g=gain*T_f                                  #New T_sky with respective gain (Beam Correction)
            Temps_g[i,j]=mean(T_f_g)                        #Mean temperature
         
    return Temps_g

## 7. Best times for observing <a class="anchor" id="TObs"></a>

We are interested in taking measurements of a mostly structureless sky. Because of this, the best times for observing will be when the center of the galaxy is not above the horizon.

The function $\textbf{obs}$ will give the hours of the day when is best to observe. We consider only the hours when we can't see from -45 to 45 degrees in galactic longitude above the horizon.

In [7]:
def obs(t0,days):
    '''Takes start date (t0) and the number of days of the observation (days).
       Returns best times for observing (dates, start time and the end time)'''
    
    #Galactic coordinates
    g0=SkyCoord(0*u.deg, 0*u.deg, frame='galactic')    #galactic center
    g6=SkyCoord(45*u.deg, 0*u.deg, frame='galactic') 
    g18=SkyCoord(-45*u.deg, 0*u.deg, frame='galactic')
    
    #Days
    t100=t0 + TimeDelta(30*60, format='sec')*linspace(0,48*days,48*days) #dt=30 min, t=#days
    altaz100 = AltAz(location=Alma_loc, obstime=t100)
    
    #Altaz coordinates
    g0_aa=g0.transform_to(altaz100)
    g6_aa=g6.transform_to(altaz100)
    g18_aa=g18.transform_to(altaz100)
    
    #Ok to observe when galaxy center below the horizon
    ok=reduce(intersect1d,(argwhere(g6_aa.alt<0),(argwhere(g0_aa.alt<0)),
                           argwhere(g18_aa.alt<0)))
    
    #Dates for observing
    from_=array([t100[ok[0]].to_datetime().hour])
    to_=array([])
    dates=array([t100[ok[0]].to_datetime()])
    for i in range(len(ok)-1):
        if ok[i+1]!=(ok[i]+1):
            from_=append(from_,t100[ok[i+1]].to_datetime().hour)
            to_=append(to_,t100[ok[i]].to_datetime().hour)
            dates=append(dates,t100[ok[i]].to_datetime())
    to_=append(to_,t100[ok[-1]])
    
    return dates,from_,to_

## 8. Galactic Coordinates of the Zenith <a class="anchor" id="GalCoord"></a>

The function $\textbf{GalCoord_Cenit}$ will give us the Galactic coordinates of the zenith at certain times.

In [8]:
def GalCoord_Cenit(Earthloc,times):
    '''Takes the Antenna location and the times for observations.
       Returns galactic coordinates for the cenit'''
    
    galcoords=array([])
    
    for i in tqdm(range (len(times))):
        
        altaz = SkyCoord(alt=90*u.deg, az =0*u.rad, obstime=times[i], frame='altaz', location=Earthloc) #Cenit AltAz Coordinates
        
        galcoords=append(galcoords,altaz.galactic) #Transform to galactic and save
    
    return galcoords

## 9. Simulation of the Sky Temperature extrapolated to a frequency <a class="anchor" id="SimTsky"></a>

The function $\textbf{Sim_Tsky}$ will give us the Sky temperature simulation with the associated coordinates extrapolated to a certain frequency from the Haslam Map.

In [1]:
def Sim_Tsky(freq,ts,alpha):  
    '''Takes frequencies (fs), times in AltAz frame (ts), the spectral index (alpha). 
       Returns the Simulation of the Sky Temperature (T_sky) extrapolated to a frequency and its coordinates (alt) and (az)'''
    
    T_Hmap_freq=T_freq(alpha,408,freq,T_Hmap)  #Extrapolation of the Haslam Map to (freq)
    
    aa_coord=g_coord.transform_to(ts)          #Galactic Coords of Haslam Map to Altaz Coords
    
    mask=argwhere(aa_coord.alt.value>-0.2)     #Only coordinates above the horizon
    
    T_sky=T_Hmap_freq[mask]                    #Sky Temperature
        
    alt=aa_coord.alt.value[mask]               #Alt coordinate
    
    az=aa_coord.az.value[mask]                 #Az coordinate
         
    return alt,az,T_sky